In [ ]:
import init
import pandas as pd
import os 
from bs4 import BeautifulSoup

import requests

from controller.url_controller import get_full_url
from controller.training_list_controller import get_target_training_list,parse_training_list_for_enterprise,parse_training_list_for_job_hunter,parse_training_list_for_worker
from const import TRAINING_FOR_WORKER,TRAINING_FOR_ENTERPRISE,TRAINING_FOR_JOB_HUNTER,INFO_TYPE,CLASS_TYPE
from Class.url import URL

api_key = os.getenv('HRD_API_KEY')

In [ ]:

# opts = {
#         "authKey": "serviceKey",
#         "returnType": "XML",  # only XML avaliable
#         "outType": "1",  # 1 - list
#         "pageNum": "1",  # maximum 1000
#         "pageSize": "100",  # maximum 100
        
#         "srchTraArea1":"", # 훈련지역 대분류
#         "srchTraArea2":"", # 훈련지역 중분류
        
#         "srchKeco1":"", # 훈련분야 대분류
#         "srchKeco2":"", # 훈련분야 중분류
#         "srchKeco3":"", # 훈련분야 소분류
        
#         "crseTracseSe":"", # 훈련유형 # 중장년특화 X
        
#         "srchTraGbn":"", # 훈련구분코드 # 중장년특화 X
#         "srchTraType":"", # 훈련종류
        
#         "sort": "ASC", 정렬방법
#         "sortCol": "TR_STT_DT",  # TOT_FXNUM - 모집인원, TR_STT_DT - 훈련시작일, TR_NM_i - 훈련과정명

#         "srchTraStDt": "19700101",  # 훈련 시작일 from
#         "srchTraEndDt": "20221231",  # 훈련 시작일 to

#         "srchTraProcessNm":"", # 훈련과정명
#         "srchTraOrganNm":"", # 훈련기관명
# }


In [ ]:
url = URL()

url.set_parameter({
    'authKey':api_key,
    "returnType": "XML", 
    "sort":'DESC',
    "outType": "1", 
    "pageNum": "1",  
    "pageSize": "100", 
    "sortCol": "TR_STT_DT"
})

In [ ]:
# 2000 01 01 / 2000 03 31
# 2000 04 01 / 2000 06 30
# 2000 07 01 / 2000 09 30
# 2000 10 01 / 2000 12 31

dates = [
    ['0101','0331'],
    ['0401','0630'],
    ['0701','0930'],
    ['1001','1231']
    ]

In [ ]:
def get_all_training_list(t:str): # 2022년 5월 30일 시작일자까지
    url_type = None
    
    if t == '구직자':
        url_type = TRAINING_FOR_JOB_HUNTER
    elif t == '근로자':
        url_type = TRAINING_FOR_WORKER
    elif t == '기업':
        url_type = TRAINING_FOR_ENTERPRISE
    else:
        return False
    
    url.set_url(url_type[INFO_TYPE['목록']])
    
    for i in range(2018,2022):
        for j in dates:
            if (i == 2022) and (j[0] == '0701'): # 기간에 맞게 고치세요.
                break
            
            url.set_parameter({
                "srchTraStDt": "".join([str(i),j[0]]),  
                "srchTraEndDt": "".join([str(i),j[1]]),
            })
            
            res = requests.get(get_full_url(url), headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
                },
            )   

            xml = BeautifulSoup(res.content,'lxml-xml')
            lst = []
            
            c = xml.find('scn_cnt').text
            cnt = int(c) // 100
            print(c,cnt)
            
            for k in range(1,cnt+2):
                url.set_parameter({
                    "pageNum":'{}'.format(k)
                })
                res = requests.get(get_full_url(url), headers=
                                {
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
                                    },
                                )   

                xml = BeautifulSoup(res.content,'lxml-xml')
                scn_list = xml.find("srchList").findAll("scn_list")
                
                lst.append(parse_training_list_for_job_hunter(scn_list))
            
            df = pd.concat(lst,axis=0,ignore_index=True)
            df['과정유형'] = CLASS_TYPE[t]
            df.to_csv(''.join(['../files/list/',t,str(i),j[0],j[1],'.csv']),index=False)

In [ ]:
get_all_training_list('구직자')

In [ ]:
get_all_training_list('근로자')

In [ ]:
get_all_training_list('기업')

In [ ]:
# # 구직자과정
    
# url.set_url(TRAINING_FOR_JOB_HUNTER[INFO_TYPE['목록']])
# for i in range(2018,2019):
#     for j in dates:
#         url.set_parameter({
#             "srchTraStDt": "".join([str(i),j[0]]),  
#             "srchTraEndDt": "".join([str(i),j[1]]),
#         })
        
#         res = requests.get(get_full_url(url), headers={
#             "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
#             },
#         )   

#         xml = BeautifulSoup(res.content,'lxml-xml')
#         lst = []
        
#         c = xml.find('scn_cnt').text
#         cnt = int(c) // 100
#         print(c,cnt)
        
#         for k in range(1,cnt+2):
#             url.set_parameter({
#                 "pageNum":'{}'.format(k)
#             })
#             res = requests.get(get_full_url(url), headers=
#                                {
#                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
#                                    },
#                                )   

#             xml = BeautifulSoup(res.content,'lxml-xml')
#             scn_list = xml.find("srchList").findAll("scn_list")
            
#             lst.append(parse_training_list_for_job_hunter(scn_list))
        
#         pd.concat(lst,axis=0,ignore_index=True).to_csv(''.join(['../files/',str(i),j[0],j[1],'.csv']),index=False)
  
        
            